   * https://debezium.io/documentation/reference/stable/connectors/sqlserver.html: Debezium connector for SQL Server
   * https://github.com/debezium/debezium-examples/tree/main/tutorial#using-sql-server
      * https://github.com/debezium/debezium-examples/blob/main/tutorial/debezium-sqlserver-init/inventory.sql

In [37]:
!kubecolor -n kafka-main-cluster get kafkaconnector debezium-connector-sqlserver

NAME                           CLUSTER      CONNECTOR CLASS                                      MAX TASKS   READY
debezium-connector-sqlserver   my-connect   io.debezium.connector.sqlserver.SqlServerConnector   1           


In [ ]:
!kubectl get -n kafka-main-cluster kafkaconnectors debezium-connector-sqlserver -o 

In [64]:
!kubectl get -n kafka-main-cluster kafkaconnectors debezium-connector-sqlserver -o jsonpath='{.status.connectorStatus.tasks[0].trace}'

org.apache.kafka.connect.errors.ConnectException: Error configuring an instance of KafkaSchemaHistory; check the logs for details
	at io.debezium.storage.kafka.history.KafkaSchemaHistory.configure(KafkaSchemaHistory.java:208)
	at io.debezium.relational.HistorizedRelationalDatabaseConnectorConfig.getSchemaHistory(HistorizedRelationalDatabaseConnectorConfig.java:137)
	at io.debezium.relational.HistorizedRelationalDatabaseSchema.<init>(HistorizedRelationalDatabaseSchema.java:49)
	at io.debezium.connector.sqlserver.SqlServerDatabaseSchema.<init>(SqlServerDatabaseSchema.java:35)
	at io.debezium.connector.sqlserver.SqlServerConnectorTask.start(SqlServerConnectorTask.java:86)
	at io.debezium.connector.common.BaseSourceTask.start(BaseSourceTask.java:141)
	at org.apache.kafka.connect.runtime.AbstractWorkerSourceTask.initializeAndStart(AbstractWorkerSourceTask.java:280)
	at org.apache.kafka.connect.runtime.WorkerTask.doRun(WorkerTask.java:202)
	at org.apache.kafka.connect.runtime.WorkerTask.run(

# Get connector errors

In [34]:
!kubectl get -n kafka-main-cluster kafkaconnectors debezium-connector-sqlserver -o jsonpath='{.status.conditions}' | jq

[
  {
    "lastTransitionTime": "2024-02-07T15:20:11.220586346Z",
    "message": "PUT /connectors/debezium-connector-sqlserver/config returned 400 (Bad Request): Connector configuration is invalid and contains the following 1 error(s):\nUser debezium does not have access to CDC schema in the following databases: testDB. This user can only be used in initial_only snapshot mode\nYou can also find the above list of errors at the endpoint `/connector-plugins/{connectorType}/config/validate`",
    "reason": "ConnectRestException",
    "status": "True",
    "type": "NotReady"
  }
]


   * https://debezium.io/documentation/reference/stable/connectors/sqlserver.html#setting-up-sqlserver
      * "Example: Enabling a SQL Server database for the CDC template"   
```sql
USE test

EXEC sys.sp_cdc_enable_db
EXEC sys.sp_cdc_help_change_data_capture
```


# SqlServer version

In [35]:
! password=$(kubectl -n sqlserver get secrets mssqlserver-2022-secret -o jsonpath="{.data.sapassword}" | base64 --decode) && \
  \
  \
  kubectl -n sqlserver run mssqlcli --image=mcr.microsoft.com/mssql-tools -ti --restart=Never --rm -- \
        /opt/mssql-tools/bin/sqlcmd -S mssqlserver-2022.sqlserver.svc -U sa -P $password -Q "select @@VERSION;"

                                                                                                                                                                                                                                                                                                            
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Microsoft SQL Server 2022 (RTM-CU11) (KB5032679) - 16.0.4105.2 (X64) 
	Nov 14 2023 18:33:19 
	Copyright (C) 2022 Microsoft Corporation
	Developer Edition (64-bit) on Linux (Ubuntu 22.04.3 LTS) <X64>                                                                                                      

(1 rows affected)
pod "mssqlcli" deleted


# Verifying that the user has access to the CDC table
   * https://debezium.io/documentation/reference/stable/connectors/sqlserver.html#_verifying_that_the_user_has_access_to_the_cdc_table

In [30]:
! password=$(kubectl -n sqlserver get secrets mssqlserver-2022-secret -o jsonpath="{.data.sapassword}" | base64 --decode) && \
  kubectl -n sqlserver run mssqlcli --image=mcr.microsoft.com/mssql-tools -ti --restart=Never --rm -- \
        /opt/mssql-tools/bin/sqlcmd -S mssqlserver-2022.sqlserver.svc -U sa -P $password -Q\
       \
       \
       "use testDB; \
        exec sys.sp_cdc_help_change_data_capture" 2> /dev/null > sp_cdc_help_change_data_capture.txt

In [52]:
! username=$(kubectl -n sqlserver get secrets  sqlserver-user-debezium -o jsonpath="{.data.username}" | base64 --decode) && \
  password=$(kubectl -n sqlserver get secrets  sqlserver-user-debezium -o jsonpath="{.data.password}" | base64 --decode) && \
  echo "User: $username Password: $password" && \
  echo "" && \
  kubectl -n sqlserver run mssqlcli --image=mcr.microsoft.com/mssql-tools -ti --restart=Never --rm -- \
        /opt/mssql-tools/bin/sqlcmd -S mssqlserver-2022.sqlserver.svc -U $username -P $password -Q \
        "use testDB; \
        select count(*) from products"
  

User: debezium Password: DeZium10#@A

Changed database context to 'testDB'.
           
-----------
          9

(1 rows affected)
pod "mssqlcli" deleted
